# 运算重载

[![下载Notebook](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/operation/mindspore_op_overload.ipynb)&emsp;
[![下载样例代码](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/operation/mindspore_op_overload.py)&emsp;
[![查看源文件](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/experts/source_zh_cn/operation/op_overload.ipynb)

`mindspore.ops.functional`模块提供了一些用户可能会用到操作，例如运算重载、取最大/小值、生成正态分布随机数、生成拉普拉斯分布随机数等，下面我们介绍运算重载的使用方式。

> 更多接口相关信息请参考[API文档](https://mindspore.cn/docs/api/en/master/api_python/mindspore.ops.html#functional)。

## 生成重载函数

`MultitypeFuncGraph`用于生成重载函数，支持不同类型的输入。用户可以使用`MultitypeFuncGraph`自定义一组重载的函数，根据不同的输入类型来定义对应的处理逻辑，这一点与C++的重载类似。

下面我们通过一段简单的示例代码来进行说明。代码样例如下：

In [3]:
import numpy as np
from mindspore.ops import MultitypeFuncGraph
from mindspore import Tensor
import mindspore.ops as ops

# 初始化MultitypeFuncGraph对象，重载函数名称为：add
add = MultitypeFuncGraph('add')

# 使用带有输入类型的 `register` 作为待注册函数的装饰器
@add.register("Number", "Number")
def add_scalar(x, y):
    """定义两个标量的加法"""
    return ops.scalar_add(x, y)

@add.register("Tensor", "Tensor")
def add_tensor(x, y):
    """定义两个张量的加法"""
    return ops.tensor_add(x, y)

tensor1 = Tensor(np.array([[1.2, 2.1], [2.2, 3.2]]).astype('float32'))
tensor2 = Tensor(np.array([[1.2, 2.1], [2.2, 3.2]]).astype('float32'))

print('scalar', add(1, 2))
print('tensor', add(tensor1, tensor2))

scalar 3
tensor [[2.4 4.2]
 [4.4 6.4]]


从上面的打印结果可以看出，经过对函数`add`重载后，其不仅可以完成两个`Number`类型的加法，而且还可以完成两个`Tensor`类型的加法。

## 批量调用重载函数

`HyperMap`可以对一组或多组输入做指定的运算，可以配合`MultitypeFuncGraph`一起使用。例如定义一组重载的`add`函数后，对多组不同类型的输入进行`add`运算。不同于`Map`，`HyperMap` 能够用于嵌套结构，对序列或嵌套序列中的输入做指定运算。

下面我们通过一段样例代码来说明：

In [5]:
from mindspore import dtype as mstype
from mindspore import Tensor
from mindspore.ops import MultitypeFuncGraph, HyperMap
import mindspore.ops as ops

# 初始化`MultitypeFuncGraph`对象，重载函数名称为：add
add = MultitypeFuncGraph('add')

# 使用带有输入类型的 `register` 作为待注册函数的装饰器
@add.register("Number", "Number")
def add_scalar(x, y):
    """定义两个标量的加法"""
    return ops.scalar_add(x, y)

@add.register("Tensor", "Tensor")
def add_tensor(x, y):
    """定义两个张量的加法"""
    return ops.tensor_add(x, y)

# 定义HyperMap的操作：add
add_map = HyperMap(add)

# 定义“被加数”List
x1 = Tensor(1, mstype.float32)
x2 = Tensor(2, mstype.float32)
x3 = 1
x_list = [x1, x2, x3]

# 定义“加数”List
y1 = Tensor(3, mstype.float32)
y2 = Tensor(4, mstype.float32)
y3 = 2
y_list = [y1, y2, y3]

output = add_map(x_list, y_list)
print("output:", output)

output: (Tensor(shape=[], dtype=Float32, value= 4), Tensor(shape=[], dtype=Float32, value= 6), 3)


上面的代码中传入`add_map`的输入包含了两个序列：`x_list`和`y_list`。`HyperMap`会以`operation(args[0][i], args[1][i])`的形式，分别从两个序列中取相应的元素作为`add`函数的输入`x`和`y`并完成相应的运算，即：`x1` + `y1`、`x2` + `y2`和`x3` + `y3`。